In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../src")

In [2]:
import numpy as np
import pandas as pd

# import fields
import model

In [7]:
sigma = model.MarginalParam("sigma", 1.0, (0.4, 3.5))
sigma.to_dataframe()

,name,value,bounds
0,sigma_11,1.0,"(0.4, 3.5)"
1,sigma_22,1.0,"(0.4, 3.5)"


In [8]:
nu = model.CrossParam("nu", 1.5, (0.2, 3.5))
nu.to_dataframe()

,name,value,bounds
0,nu_11,1.5,"(0.2, 3.5)"
1,nu_12,1.5,"(0.2, 3.5)"
2,nu_22,1.5,"(0.2, 3.5)"


In [9]:
rho = model.RhoParam("rho", 0.0, (-1, 1))
rho.to_dataframe()

,name,value,bounds
0,rho_12,0.0,"(-1, 1)"
